In [46]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/exams.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [47]:
# For Gender: unprivileged group: female; priviliged group: male;
# For Race: unprivileged group: Group A B % E; priviliged group: Group C and D;
# Fairness metrics:
# Statistical Parity Difference
# Disparate Impact
# If math score >= 60 and writing score >= 60, this will be the favorable outcome; if math score < 60 and writing score < 60, this will be the unfavorable outcome.

male_math_favorable, female_math_favorable, male_writing_favorable, female_writing_favorable, male_math_unfavorable, female_math_unfavorable, male_writing_unfavorable, female_writing_unfavorable = 0, 0, 0, 0, 0, 0, 0, 0
groupCD_math_favorable, groupABE_math_favorable, groupCD_writing_favorable, groupABE_writing_favorable, groupCD_math_unfavorable, groupABE_math_unfavorable, groupCD_writing_unfavorable, groupABE_writing_unfavorable = 0, 0, 0, 0, 0, 0, 0, 0


In [48]:
for index, row in df.iterrows():
  if row['gender'] == 'male':
    if row['math score'] >= 60:
      male_math_favorable += 1
    else:
      male_math_unfavorable += 1
    if row['writing score'] >= 60:
      male_writing_favorable += 1
    else:
      male_writing_unfavorable += 1
  else:
    if row['math score'] >= 60:
      female_math_favorable += 1
    else:
      female_math_unfavorable += 1
    if row['writing score'] >= 60:
      female_writing_favorable += 1
    else:
      female_writing_unfavorable += 1
  if row['race/ethnicity'] == 'group C' or row['race/ethnicity'] == 'group D':
    if row['math score'] >= 60:
      groupCD_math_favorable += 1
    else:
      groupCD_math_unfavorable +=1
    if row['writing score'] >= 60:
      groupCD_writing_favorable += 1
    else:
      groupCD_writing_unfavorable +=1
  else:
    if row['math score'] >= 60:
      groupABE_math_favorable += 1
    else:
      groupABE_math_unfavorable +=1
    if row['writing score'] >= 60:
      groupABE_writing_favorable += 1
    else:
      groupABE_writing_unfavorable +=1

In [49]:
og_gender_math_disparate_impact = (female_math_favorable / (female_math_favorable + female_math_unfavorable)) / (male_math_favorable / (male_math_favorable + male_math_unfavorable))
og_gender_writing_disparate_impact = (female_writing_favorable / (female_writing_favorable + female_writing_unfavorable)) / (male_writing_favorable / (male_writing_favorable + male_writing_unfavorable))
og_race_math_disparate_impact = (groupABE_math_favorable / (groupABE_math_favorable + groupABE_math_unfavorable)) / (groupCD_math_favorable / (groupCD_math_favorable + groupCD_math_unfavorable))
og_race_writing_disparate_impact = (groupABE_writing_favorable / (groupABE_writing_favorable + groupABE_writing_unfavorable)) / (groupCD_writing_favorable / (groupCD_writing_favorable + groupCD_writing_unfavorable))

og_gender_math_statistical_parity = (female_math_favorable / (female_math_favorable + female_math_unfavorable)) - (male_math_favorable / (male_math_favorable + male_math_unfavorable))
og_gender_writing_statistical_parity = (female_writing_favorable / (female_writing_favorable + female_writing_unfavorable)) - (male_writing_favorable / (male_writing_favorable + male_writing_unfavorable))
og_race_math_statistical_parity = (groupABE_math_favorable / (groupABE_math_favorable + groupABE_math_unfavorable)) - (groupCD_math_favorable / (groupCD_math_favorable + groupCD_math_unfavorable))
og_race_writing_statistical_parity = (groupABE_writing_favorable / (groupABE_writing_favorable + groupABE_writing_unfavorable)) - (groupCD_writing_favorable / (groupCD_writing_favorable + groupCD_writing_unfavorable))

original_metrics = [og_gender_math_disparate_impact,
                   og_gender_writing_disparate_impact,
                   og_race_math_disparate_impact,
                   og_race_writing_disparate_impact,
                   og_gender_math_statistical_parity,
                   og_gender_writing_statistical_parity,
                   og_race_math_statistical_parity,
                   og_race_writing_statistical_parity]
independent_lables = ['gender', 'gender', 'race', 'race', 'gender', 'gender', 'race', 'race']
dependent_lables = ['math score', 'writing score', 'math score', 'writing score', 'math score', 'writing score', 'math score', 'writing score']
metric_names = ['disparate impact', 'disparate impact', 'disparate impact', 'disparate impact', 'statistical parity', 'statistical parity', 'statistical parity', 'statistical parity']

og_summary_table = list(zip(independent_lables, dependent_lables, metric_names, original_metrics))
og_summary_df = pd.DataFrame(og_summary_table, columns = ['Indepedent Variable', 'Dependent Variable', 'Fairness Metric', 'Original Metric Value'])
og_summary_df.head(10)

,Indepedent Variable,Dependent Variable,Fairness Metric,Original Metric Value
0,gender,math score,disparate impact,0.875108
1,gender,writing score,disparate impact,1.269594
2,race,math score,disparate impact,1.009801
3,race,writing score,disparate impact,0.994672
4,gender,math score,statistical parity,-0.095144
5,gender,writing score,statistical parity,0.175661
6,race,math score,statistical parity,0.006979
7,race,writing score,statistical parity,-0.003941


In [ ]:
# Disparate Metrics
# Values less than 1 indicate that facet a has a higher proportion of predicted positive outcomes than facet d. This is referred to as positive bias.
# A value of 1 indicates demographic parity.
# Values greater than 1 indicate that facet d has a higher proportion of predicted positive outcomes than facet a. This is referred to as negative bias.


# statistical parity difference
# Under 0: Higher benefit for the monitored group.
# At 0: Both groups have equal benefit.
# Over 0 Implies higher benefit for the reference group.

In [50]:
!pip install aif360
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import BinaryLabelDataset

In [52]:
conditions = [
    (df['race/ethnicity'] == 'group C') | (df['race/ethnicity'] == 'group D'),
    (df['race/ethnicity'] == 'group A') | (df['race/ethnicity'] == 'group B') | (df['race/ethnicity'] == 'group E')]
choices = [0, 1]
df['race'] = np.select(conditions, choices)
df['gender_binary'] = np.where(df['gender']=='male', 0, 1)
df['math_score_binary'] = np.where(df['math score'] >= 60, 0, 1)
df['writing_score_binary'] = np.where(df['writing score'] >= 60, 0, 1)


In [53]:
df2 = df[['gender_binary', 'race', 'math_score_binary', 'writing_score_binary']].copy()

In [54]:
dataset = BinaryLabelDataset(favorable_label=0,
                             unfavorable_label=1,
                             df=df2,
                             label_names=['math_score_binary'],
                             protected_attribute_names=['gender_binary', 'race'],
                             privileged_protected_attributes=[[0, 0], [0, 0]])

In [55]:
RW = Reweighing(unprivileged_groups=[{'gender_binary': 1, 'race': 1}], privileged_groups=[{'gender_binary': 0, 'race': 0}])
dataset_reweighted = RW.fit_transform(dataset)
df_reweighted, _ = dataset_reweighted.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)


In [56]:
gender_grouped = df_reweighted.groupby('gender_binary')[['math_score_binary', 'writing_score_binary']].mean()
race_grouped = df_reweighted.groupby('race')[['math_score_binary', 'writing_score_binary']].mean()

In [57]:
gender_math_disparate_impact = gender_grouped.loc[1, 'math_score_binary'] / gender_grouped.loc[0, 'math_score_binary']
race_math_disparate_impact = race_grouped.loc[1, 'math_score_binary'] / race_grouped.loc[0, 'math_score_binary']

gender_writing_disparate_impact = gender_grouped.loc[1, 'writing_score_binary'] / gender_grouped.loc[0, 'writing_score_binary']
race_writing_disparate_impact = race_grouped.loc[1, 'writing_score_binary'] / race_grouped.loc[0, 'writing_score_binary']


gender_math_spd = gender_grouped.loc[1, 'math_score_binary'] - gender_grouped.loc[0, 'math_score_binary']
race_math_spd = race_grouped.loc[1, 'math_score_binary'] - race_grouped.loc[0, 'math_score_binary']
gender_writing_spd = gender_grouped.loc[1, 'writing_score_binary'] - gender_grouped.loc[0, 'writing_score_binary']
race_writing_spd = race_grouped.loc[1, 'writing_score_binary'] - race_grouped.loc[0, 'writing_score_binary']



new_metrics = [gender_math_disparate_impact,
                   gender_writing_disparate_impact,
                   race_math_disparate_impact,
                   race_writing_disparate_impact,
                   gender_math_spd,
                   gender_writing_spd,
                   race_math_spd,
                   race_writing_spd]


og_summary_df['New Metric Value'] = new_metrics
og_summary_df.head(10)


,Indepedent Variable,Dependent Variable,Fairness Metric,Original Metric Value,New Metric Value
0,gender,math score,disparate impact,0.875108,1.399449
1,gender,writing score,disparate impact,1.269594,0.495843
2,race,math score,disparate impact,1.009801,0.975763
3,race,writing score,disparate impact,0.994672,1.015137
4,gender,math score,statistical parity,-0.095144,0.095144
5,gender,writing score,statistical parity,0.175661,-0.175661
6,race,math score,statistical parity,0.006979,-0.006979
7,race,writing score,statistical parity,-0.003941,0.003941
